Code provided by Provilkov_Ivan

In [1]:
from tqdm import tqdm_notebook
import re
import time

PATH_TO_DATA = 'datasets/OpenSubtitlesv2018/en-ru.tmx'
PATH_TO_TRAIN = 'datasets/OpenSubtitlesv2018/'
PATH_TO_BPE = 'datasets/OpenSubtitlesv2018/'

In [2]:
%%time

en_list = []
ru_list = []
counter = 0
start = time.time()
regex_corner = re.compile('[<>?]')

with open(PATH_TO_DATA, 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        if ":lang=\"en\"" in line:
            good_tokens = regex_corner.sub(' ', line).split()[3:-2]
            en_list.append(' '.join(good_tokens))
            
        elif ":lang=\"ru\"" in line:
            good_tokens = regex_corner.sub(' ', line).split()[3:-2]
            ru_list.append(' '.join(good_tokens)) 

CPU times: user 4min 41s, sys: 8.96 s, total: 4min 50s
Wall time: 4min 59s


In [3]:
len(en_list)

21223864

In [4]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [5]:
%%time

with open(PATH_TO_TRAIN + '/train.en', 'w') as f:
    for sent in en_list:
        f.write(' '.join(tokenizer.tokenize(sent)) + '\n')

In [6]:
with open(PATH_TO_TRAIN + '/train.ru', 'w') as f:
    for sent in ru_list:
        f.write(' '.join(tokenizer.tokenize(sent)) + '\n')

In [7]:
# !pip install subword-nmt
from utils.text_preprocessing import create_train_bpe

In [8]:
%%time
create_train_bpe(PATH_TO_BPE)

Learning BPE...
Writing train files...
Learning BPE...
Writing train files...


## Test

In [9]:
TRAIN_DIR = 'datasets/OpenSubtitlesv2018/'

In [10]:
firstn=10000000

In [11]:
en_lines = []
ru_lines = []
with open(TRAIN_DIR + 'train.bpe.ru', 'r') as f:
    for i, line in enumerate(f):
        ru_lines.append(line)
        if i > firstn:
            break
        
with open(TRAIN_DIR + 'train.bpe.en', 'r') as f:
    for i, line in enumerate(f):
        en_lines.append(line)
        if i > firstn:
            break        

In [12]:
len(en_lines)

10000002

In [13]:
#data_out = np.array(ru_lines[:firstn])
#data_inp = np.array(en_lines[:firstn])

In [14]:
#data_inp.shape, data_out.shape

In [15]:
import gc
gc.collect()

0

In [16]:
from sklearn.model_selection import train_test_split
train_inp, dev_inp, train_out, dev_out = train_test_split(en_lines, ru_lines, test_size=10000,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')

inp: Looks like they did . DA@@ PH@@ N@@ E :

out: Похоже , им это удалось .


inp: Bill Carson !

out: Билл Кар@@ сон !


inp: She knows every corner of that room .

out: Она знает каждый угол в той комнате .




In [17]:
len(train_inp)

9990002

In [19]:
from utils.utils import Vocab
inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

In [22]:
class BasicModel:
    def __init__(self, name, inp_voc, out_voc, emb_size=64, hid_size=128):
        """
        A simple encoder-decoder model
        """
        self.name, self.inp_voc, self.out_voc = name, inp_voc, out_voc

        with tf.variable_scope(name):
            self.emb_inp = L.Embedding(len(inp_voc), emb_size)
            self.emb_out = L.Embedding(len(out_voc), emb_size)
            self.enc0 = tf.nn.rnn_cell.GRUCell(hid_size)

            self.dec_start = L.Dense(hid_size)
            self.dec0 = tf.nn.rnn_cell.GRUCell(hid_size)
            self.logits = L.Dense(len(out_voc))

            # prepare to translate_lines
            self.inp = tf.placeholder('int32', [None, None])
            self.initial_state = self.prev_state = self.encode(self.inp)
            self.prev_tokens = tf.placeholder('int32', [None])
            self.next_state, self.next_logits = self.decode(
                self.prev_state, self.prev_tokens)

        self.weights = tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES, scope=name)

    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :returns: initial decoder state tensors, one or many
        """
        inp_lengths = infer_length(inp, self.inp_voc.eos_ix)
        inp_emb = self.emb_inp(inp)
        with tf.variable_scope('enc0'):
            _, enc_last = tf.nn.dynamic_rnn(
                self.enc0,
                inp_emb,
                sequence_length=inp_lengths,
                dtype=inp_emb.dtype)
        dec_start = self.dec_start(enc_last)
        return [dec_start]

    def decode(self, prev_state, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits for next tokens
        :param prev_state: a list of previous decoder state tensors
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch, n_tokens]
        """
        [prev_dec] = prev_state
        prev_emb = self.emb_out(prev_tokens[:, None])[:, 0]
        with tf.variable_scope('dec0'):
            new_dec_out, new_dec_state = self.dec0(prev_emb, prev_dec)
        output_logits = self.logits(new_dec_out)
        return [new_dec_state], output_logits

    def translate_lines(self, inp_lines, sess, inp_voc, out_voc, max_len=100):
        """
        Translates a list of lines by greedily selecting most likely next token at each step
        :returns: a list of output lines, a sequence of model states at each step
        """
        state = sess.run(self.initial_state,
                         {self.inp: inp_voc.to_matrix(inp_lines)})
        outputs = [[self.out_voc.bos_ix] for _ in range(len(inp_lines))]
        all_states = [state]
        finished = [False] * len(inp_lines)

        for t in range(max_len):
            state, logits = sess.run(
                [self.next_state, self.next_logits], {
                    **dict(zip(self.prev_state, state)), self.prev_tokens:
                    [out_i[-1] for out_i in outputs]
                })
            next_tokens = np.argmax(logits, axis=-1)
            all_states.append(state)
            for i in range(len(next_tokens)):
                outputs[i].append(next_tokens[i])
                finished[i] |= next_tokens[i] == self.out_voc.eos_ix
        return out_voc.to_lines(outputs), all_states

In [35]:
class AttentionLayer:
    def __init__(self, name, enc_size, dec_size, hid_size, activ=tf.tanh,):
        """ A layer that computes additive attention response and weights """
        self.name = name
        self.enc_size = enc_size # num units in encoder state
        self.dec_size = dec_size # num units in decoder state
        self.hid_size = hid_size # attention layer hidden units
        self.activ = activ       # attention layer hidden nonlinearity

        with tf.variable_scope(name):
            # Tensorflow can define input sizes by itself
            self.enc_att = L.Dense(self.hid_size)
            self.dec_att = L.Dense(self.hid_size)
            self.out_att = L.Dense(self.hid_size)

    def __call__(self, enc, dec, inp_mask):
        """
        Computes attention response and weights
        :param enc: encoder activation sequence, float32[batch_size, ninp, enc_size]
        :param dec: single decoder state used as "query", float32[batch_size, dec_size]
        :param inp_mask: mask on enc activatons (0 after first eos), float32 [batch_size, ninp]
        :returns: attn[batch_size, enc_size], probs[batch_size, ninp]
            - attn - attention response vector (weighted sum of enc)
            - probs - attention weights after softmax
        """
        with tf.variable_scope(self.name):
            # Compute logits
            # We will use multiple logits for each attention sentence, may be it will improve score.
            a = self.out_att(self.activ(self.enc_att(enc) + tf.expand_dims(self.dec_att(dec),1)))
            # Apply mask - if mask is 0, logits should be -inf or -1e9
            # You may need tf.where
            a = tf.where(inp_mask, a, tf.fill(tf.shape(a), -1e9))
            # Compute attention probabilities (softmax)
            
            probs = tf.nn.softmax(a, axis=1)
            
            # Compute attention response using enc and probs
            attn = tf.reduce_sum(tf.multiply(probs, a), axis=1)
            
            return attn, probs
        

In [36]:
class SpearTranslate(BasicModel):
    def __init__(self, name, inp_voc, out_voc, config,
                 emb_size=64, hid_size=128, attn_size=128):
        """ Translation model that uses attention. See instructions above. """
        self.name = name
        self.inp_voc = inp_voc
        self.out_voc = out_voc
        self.hid_size=hid_size

        with tf.variable_scope(name):
            
            # define model layers
            self.emb_inp = L.Embedding(len(inp_voc), emb_size)
            self.emb_out = L.Embedding(len(out_voc), emb_size)
            with tf.variable_scope("forward"):
                self.enc0 = tf.nn.rnn_cell.LSTMCell(hid_size, forget_bias=1.0, state_is_tuple=False)
            with tf.variable_scope("backward"):
                self.enc1 = tf.nn.rnn_cell.LSTMCell(hid_size, forget_bias=1.0, state_is_tuple=False)
                
            self.attention = AttentionLayer('Attention', hid_size *4, hid_size, hid_size)
            self.dec_start = L.Dense(hid_size*2)
            
            self.dec0 = tf.nn.rnn_cell.LSTMCell(hid_size, forget_bias=1.0, state_is_tuple=False)
            self.logits = L.Dense(len(out_voc))
            
            # prepare to translate_lines
            self.inp = tf.placeholder('int32', [None, None])
            self.initial_state = self.prev_state = self.encode(self.inp)
            self.prev_tokens = tf.placeholder('int32', [None])
            self.next_state, self.next_logits = self.decode(self.prev_state, self.prev_tokens)

        self.weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=name)

    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :return: a list of initial decoder state tensors
        """
        
        # encode input sequence, create initial decoder states
        inp_lengths = infer_length(inp, self.inp_voc.eos_ix)
        mask = tf.expand_dims(tf.cast(infer_mask(inp, self.inp_voc.eos_ix), tf.bool), axis=2)

        inp_mask = tf.tile(mask, (1, 1, self.hid_size))
        #inp_mask = tf.cast(infer_mask(inp, self.inp_voc.eos_ix), tf.bool)
        inp_emb = self.emb_inp(inp)
        
        with tf.variable_scope('enc0'):
            enc_outputs_t, enc_last_t = tf.nn.bidirectional_dynamic_rnn(
                              self.enc0, self.enc1, inp_emb,
                              sequence_length=inp_lengths,
                              dtype=inp_emb.dtype)
            #[batch, time, hid_size*4]
            enc_outputs = tf.concat(enc_outputs_t, 2)
            #[batch, hid_size * 4]
            enc_last = tf.concat(enc_last_t, 1)
            
        dec_start = self.dec_start(enc_last)
        
        # apply attention layer from initial decoder hidden state
        attn, first_attn_probas = self.attention(enc_outputs, dec_start, inp_mask)
        
        # Build first state: include
        # * initial states for decoder recurrent layers
        # * encoder sequence and encoder attn mask (for attention)
        # * make sure that last state item is attention probabilities tensor
        
        first_state = [dec_start, enc_outputs, inp_mask, first_attn_probas]
        return first_state

    def decode(self, prev_state, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits
        :param prev_state: a list of previous decoder state tensors
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch,n_tokens]
        """
        # Unpack your state: you will get tensors in the same order that you've packed in encode
        [prev_dec, enc_outputs, inp_mask, first_attn_probas] = prev_state
        
        
        # Perform decoder step
        # * predict next attn response and probas given previous decoder state
        # * use prev tokens and next attn response to update decoder states
        # * predict logits
        prev_emb = self.emb_out(prev_tokens[:,None])[:,0]
        
        next_attn_response, next_attn_probas = self.attention(enc_outputs, prev_dec, inp_mask)
        
        with tf.variable_scope('dec0'):
            dec_input = tf.concat([prev_emb, next_attn_response], axis=-1)
            new_dec_out, new_dec_state = self.dec0(dec_input, prev_dec)
        
        output_logits = self.logits(new_dec_out)
        
        # Pack new state:
        # * replace previous decoder state with next one
        # * copy encoder sequence and mask from prev_state
        # * append new attention probas
        
        next_state = [new_dec_state, enc_outputs, inp_mask, next_attn_probas]
        return next_state, output_logits

In [37]:
class Config:
    pass

In [43]:
def infer_length(seq, eos_ix, time_major=False, dtype=tf.int32):
    """
    compute length given output indices and eos code
    :param seq: tf matrix [time,batch] if time_major else [batch,time]
    :param eos_ix: integer index of end-of-sentence token
    :returns: lengths, int32 vector of shape [batch]
    """
    axis = 0 if time_major else 1
    is_eos = tf.cast(tf.equal(seq, eos_ix), dtype)
    count_eos = tf.cumsum(is_eos,axis=axis,exclusive=True)
    lengths = tf.reduce_sum(tf.cast(tf.equal(count_eos,0),dtype),axis=axis)
    return lengths


def infer_mask(seq, eos_ix, time_major=False, dtype=tf.float32):
    """
    compute mask given output indices and eos code
    :param seq: tf matrix [time,batch] if time_major else [batch,time]
    :param eos_ix: integer index of end-of-sentence token
    :returns: mask, float32 matrix with '0's and '1's of same shape as seq
    """
    axis = 0 if time_major else 1
    lengths = infer_length(seq, eos_ix, time_major=time_major)
    mask = tf.sequence_mask(lengths, maxlen=tf.shape(seq)[axis], dtype=dtype)
    if time_major: mask = tf.transpose(mask)
    return mask




In [44]:
config = Config()

In [45]:
#config.enc_dim=64
#config.dec_dim=64
#config.dec_layer_size = 3

In [46]:
import tensorflow as tf
import keras.layers as L

In [47]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [48]:
model = SpearTranslate('Spear', inp_voc, out_voc, config, emb_size=100, hid_size=256, attn_size=256)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


In [52]:
def compute_loss(model, inp, out, out_voc, **flags):
    """
    Compute loss (float32 scalar) as in the formula above
    :param inp: input tokens matrix, int32[batch, time]
    :param out: reference tokens matrix, int32[batch, time]
    
    In order to pass the tests, your function should
    * include loss at first EOS but not the subsequent ones
    * divide sum of losses by a sum of input lengths (use infer_length or infer_mask)
    """
    mask = infer_mask(out, out_voc.eos_ix)    
    logits_seq = compute_logits(model, inp, out, **flags)
    
    # Compute loss as per instructions above
    print(mask.shape)
    print(logits_seq.shape)
    sparce_softmax = tf.losses.sparse_softmax_cross_entropy(labels=out, logits=logits_seq)
    return tf.reduce_sum(sparce_softmax * mask) / tf.reduce_sum(mask)

def compute_bleu(model, inp_lines, out_lines, sess, inp_voc, out_voc, bpe_sep='@@ ', **flags):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations, _ = model.translate_lines(inp_lines, sess, inp_voc, out_voc, **flags)
    # Note: if you experience out-of-memory error, split input lines into batches and translate separately
    return corpus_bleu([[ref] for ref in out_lines], translations) * 100

def compute_logits(model, inp, out, **flags):
    """
    :param inp: input tokens matrix, int32[batch, time]
    :param out: reference tokens matrix, int32[batch, time]
    :returns: logits of shape [batch, time, voc_size]
    
    * logits must be a linear output of your neural network.
    * logits [:, 0, :] should always predic BOS
    * logits [:, -1, :] should be probabilities of last token in out
    This function should NOT return logits predicted when taking out[:, -1] as y_prev
    """
    batch_size = tf.shape(inp)[0]
    
    # Encode inp, get initial state
    first_state = model.encode(inp, **flags)
    
    # initial logits: always predict BOS
    first_logits = tf.log(tf.one_hot(tf.fill([batch_size], model.out_voc.bos_ix),
                                     len(model.out_voc)) + 1e-30)
    
    
    # Decode step
    def step(blob, y_prev):
        # Given previous state, obtain next state and next token logits
        # prev_tokens int vector of batch size??
        # return state and logits
        prev_state, logits = blob
        state, logits = model.decode(prev_state, y_prev)
        return [state, logits]
      
    # Decode step
    #def step(prev_state, y_prev):
        # Given previous state, obtain next state and next token logits
        # prev_tokens int vector of batch size??
        # return state and logits

    #    return model.decode(prev_state, y_prev)

    # You can now use tf.scan to run step several times.
    # use tf.transpose(out) as elems (to process one time-step at a time)
    # docs: https://www.tensorflow.org/api_docs/python/tf/scan
    
    
    _, logits_seq = tf.scan(step, 
           elems=tf.transpose(out)[:-1],
           initializer=[first_state, first_logits])
    
    # prepend first_logits to logits_seq
    logits_seq = tf.concat((first_logits[None], logits_seq), axis=0)
    
    # Make sure you convert logits_seq from [time, batch, voc_size] to [batch, time, voc_size]
    logits_seq = tf.transpose(logits_seq, [1, 0, 2])
    
    return logits_seq

In [53]:
inp = tf.placeholder('int32', [None, None])
out = tf.placeholder('int32', [None, None])

loss = compute_loss(model, inp, out, out_voc)
train_step = tf.train.AdamOptimizer().minimize(loss)

(?, ?)
(?, ?, 10701)
Instructions for updating:
Use tf.cast instead.


In [54]:
tf.summary.scalar("train_loss", loss)

<tf.Tensor 'train_loss:0' shape=() dtype=string>

In [55]:
metrics = {'train_loss': [], 'dev_bleu': []}
sess.run(tf.global_variables_initializer())

In [56]:
len(train_inp)

9990002

In [57]:
from tqdm import tqdm_notebook, trange
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pickle

In [58]:
EPOCH = 5
INP_SIZE = len(train_inp)

In [59]:
batch_size = 128

In [60]:
saver = tf.train.Saver()

In [61]:
train_writer = tf.summary.FileWriter('./tmp/tensorboard/')

In [62]:
merged = tf.summary.merge_all()

In [63]:
summary_n = tf.Summary()

In [64]:
for i in range(EPOCH):
    #while cur_ix < INP_SIZE:
    cur_ix = 0
    for _ in trange(INP_SIZE//batch_size):
        step = i * INP_SIZE + cur_ix
        feed_dict = {
            inp: inp_voc.to_matrix(train_inp[cur_ix: cur_ix+batch_size]),
            out: out_voc.to_matrix(train_out[cur_ix: cur_ix+batch_size]),
        }
        cur_ix += batch_size
        
        loss_t, _, summary = sess.run([loss, train_step, merged], feed_dict)
        train_writer.add_summary(summary, step)
        metrics['train_loss'].append((step, loss_t))
        if cur_ix % 500 == 5:
            bleus = []
            cur_dev_ix = 0
            dev_len = len(dev_inp)
            while cur_dev_ix < dev_len:
                bleus.append(compute_bleu(model, 
                                          dev_inp[cur_dev_ix:cur_dev_ix+batch_size], 
                                          dev_out[cur_dev_ix:cur_dev_ix+batch_size], 
                                          sess, inp_voc, out_voc))
                cur_dev_ix += batch_size
            
            metrics['dev_bleu'].append((step, np.mean(bleus)))
           
            clear_output(True)
            plt.figure(figsize=(12,4))
            for i, (name, history) in enumerate(sorted(metrics.items())):
                plt.subplot(1, len(metrics), i + 1)
                plt.title(name)
                plt.plot(*zip(*history))
                plt.grid()
            plt.show()
            print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-100:], axis=0)[1], flush=True)
            print("EPOCH: ", i)
            # Tensorboard
            summary_n.value.add(tag=tagname, simple_value=metrics['dev_bleu'][-1])
            train_writer.add_summary(summary_n, step)
            train_writer.flush()
        if cur_ix % 1000 == 0:
            saver.save(sess, "/tmp/model1.ckpt")
            with open('/tmp/metrics.pkl', 'wb') as f:
                pickle.dump(metrics, f)

  0%|          | 2/78046 [00:18<245:53:07, 11.34s/it]

KeyboardInterrupt: 